In [ ]:
import os
from pathlib import Path
from ipyfilechooser import FileChooser

from eeg_preprocessing.preprocessing import *
from eeg_preprocessing.utils.io_raw import read_raw, create_epochs_from_raw
from eeg_preprocessing.utils.montage import set_raw_montage_from_locs

from matplotlib import pyplot as plt
%matplotlib qt

In [ ]:
# Set base path to EEG data
base_path = '/Volumes/raw_data_EEG/'

# Create folder for preprocessed and interim files
folder_name = 'preprocessed'
interim_path = os.path.join(base_path, folder_name)

# Use the widget to navigate to the experiment folder path and select an EEG file 
fc = FileChooser(base_path)
fc.filter_pattern = ['*.edf']

display(fc)

In [ ]:
# Load selected file
raw = read_raw(raw_file_path=fc.selected, add_info=False)
print(raw.info)

In [ ]:
# Set montage to raw
montage_file_path = 'Starstim32.locs'
raw = set_raw_montage_from_locs(montage_file_path=montage_file_path, raw=raw)

In [ ]:
epochs = create_epochs_from_raw(raw=raw)

In [ ]:
epochs_faster = prepare_epochs_for_ica(epochs=epochs)

In [ ]:
ica = run_ica(epochs=epochs_faster)

In [ ]:
# Visualize components on epochs

# Exclude components by selecting them, right click on component name to visulize source:
ica.plot_sources(epochs_faster, start=0, stop=10)

In [ ]:
# After selecting the components to exclude, apply ICA to epochs
# Document the number of excluded components
ica.apply(epochs_faster)
epochs_faster.info['description'] = f'n_components: {len(ica.exclude)}'

In [ ]:
epochs_faster.plot(n_epochs=10, scalings={'eeg': 20e-6}, title=raw.info['fid'])

In [ ]:
# Create path to epoch files
interim_epochs_path = os.path.join(interim_path, 'epochs')
if not os.path.exists(interim_epochs_path):
    os.makedirs(interim_epochs_path)

# Save ICA cleaned epochs 
fid = epochs_faster.info['fid']
epochs_clean_fname = f'{fid}_ICA'
postfix = '-epo.fif.gz'
epochs_faster.save(os.path.join(interim_epochs_path, f'{epochs_clean_fname}{postfix}'), overwrite=True)

In [ ]:
ar = run_autoreject(epochs_faster, n_jobs=11, subset=False)

In [ ]:
# Drop bad epochs (stage 1)

reject_log = ar.get_reject_log(epochs_faster)

epochs_autoreject = epochs_faster.copy().drop(reject_log.bad_epochs, reason='AUTOREJECT')

In [ ]:
epochs_ransac = run_ransac(epochs_autoreject)

In [ ]:
epochs_ransac.set_eeg_reference()

In [ ]:
epochs_ransac.plot(n_epochs=10, scalings={'eeg': 20e-6}, title=raw.info['fid'])